# Input Data Quality Check

Dans ce fichier nous allons vérifier que les données d'entrée sont correctement formatées pour leur utilisation dans DELTA.

Il s'agit d'un fichier .csv (format anglo-saxon : "," pour séparer les colonnes) qui doit suivre plusieurs spécifications vérifiées dans le script qui suit.

## 1. Déclaration des variables, chargement des données

In [1]:
import os
import pandas as pd
import re

COLOR_PRINT_RED_START = "\x1b[31m"
COLOR_PRINT_RED_END = "\x1b[31m"

En premier lieu, on va commencer par indiquer le chemin vers le fichier de données.

La variable `EXTENDED_DATASET_MODE` permet de choisir si le dataset que l'on charge est un dataset *extended* (`EXTENDED_DATASET_MODE=True`) ou standard (`EXTENDED_DATASET_MODE=False`).
- En mode standard, les données d'annotations sont uniquement les labels textuelles (format original présenté par SEBIA) ;
- En mode *extended*, les données d'annotations (y) sont les *maps de segmentation*.

In [2]:
# on précise si on souhaite avoir les données en mode "extended"
EXTENDED_DATASET_MODE = True

# si on veut le fichier de données "classique"
if not EXTENDED_DATASET_MODE:
    data_file_path = r"C:\Users\admin\Documents\Capillarys\data\2021\ifs\formatted_data\angers_dataset_v1.csv"

# si on veut le fichier de données "extended" (i.e. : y = maps de segmentation)
if EXTENDED_DATASET_MODE:
    data_file_path = r"C:\Users\admin\Documents\Capillarys\data\2021\ifs\formatted_data\angers_dataset_extended_v1.csv"

In [3]:
# on charge le dataset, en mode "string" pour récupérer EXACTEMENT les données telles qu'elles sont écrites dans le fichier .csv
raw_dataset = pd.read_csv(data_file_path, index_col=False, dtype=str)

## 2. Vérification des données

Dans les parties qui suivent, les différents QC sont réalisés. A chaque fois, l'output de la cellule s'affiche en noir si tous les QC s'exécutent correctement, ou en rouge en cas d'erreur.

**Si des items s'affichent en <font color='red'>rouge</font> dans l'output lors de l'exécution d'une cellule, c'est qu'au moins une erreur a été détectée lors du QC !**

### 2.1. Quality check basique

In [4]:
# On affiche le nombre de colonnes : c'est le nombre de courbes contenues dans le tableau + 1 (une colonne pour les indices des points)
print("Le dataset contient {} colonnes (nombre d'entrées + 1)".format(raw_dataset.shape[1]))

# On vérifie qu'il y a bien 301 lignes : une ligne pour chaque point de la courbe (300) + la ligne des labels
# La première *ligne* du tableau .csv contient les noms des colonnes et n'est donc pas comptabilisée ici
if raw_dataset.shape[0] != 301:
    print(COLOR_PRINT_RED_START + "Le dataset contient {} lignes ! Attendu : 301 lignes".format(raw_dataset.shape[0]) + COLOR_PRINT_RED_END)
else:
    print("Le dataset contient {} lignes (comme attendu)".format(raw_dataset.shape[0]))
    
if raw_dataset.iloc[0,0] != "Labels":
    print(COLOR_PRINT_RED_START + "La première valeur de la première colonne (position [0,0]) doit être == \"Labels\"" + COLOR_PRINT_RED_END)

Le dataset contient 19834 colonnes (nombre d'entrées + 1)
Le dataset contient 301 lignes (comme attendu)


### 2.2. Quality check pour les noms de colonnes

1. Vérification du nom de la première colonne
2. Vérification du nombre de colonnes
3. Vérification du nom des colonnes
4. Vérification de la cohérence nombre/nom des colonnes & échantillons du tableau

In [5]:
print("Start columns QC")

# La première colonne doit avoir pour nom : " "
if raw_dataset.columns[0] != " ":
    print(COLOR_PRINT_RED_START + "La première colonne devrait avoir un nom vide (== \" \") mais contient : <{}>".format(raw_dataset.columns[0]) + COLOR_PRINT_RED_END)

# Le nombre de colonnes (-1 pour la première colonne des indices des points des courbes)
# doit être un multiple de 11 : ELP, G, A, M, k, l, G-y, A-y, M-y, k-y, l-y pour chaque échantillon
if (raw_dataset.shape[1]-1) % 11 != 0:
    print(COLOR_PRINT_RED_START + "Le nombre de colonnes - 1 devrait être un multiple de 11 (ex : 12 colonnes au total dans le tableau)" + COLOR_PRINT_RED_END)
else:
    print("Le jeu de données contient {} échantillons".format((raw_dataset.shape[1]-1)//11))

# On doit vérifier que chaque nom de colonne matche bien la règle : "<ID-échantillon>-<ID-piste>" avec
# <ID-piste> == soit ELP, G, A, M, K ou L, G-y, A-y, M-y, K-y, L-y

# on définit le pattern regex auquel doivent obénir tous les noms de colonnes des pistes échantillons
column_regex_pattern = "^([0-9a-zA-Z]+)-(ELP|G|A|M|K|L)(-y)?$"

# puis on le checke pour chaque nom de colonne
sample_columns = raw_dataset.columns[1:].tolist()
for c,sample_column in enumerate(sample_columns): # pour chaque nom de colonne
    if re.match(column_regex_pattern, sample_column) is None: # check regex
        print(COLOR_PRINT_RED_START + "Colonne à l'index {} : nom de colonne incorrect : <{}>".format(c,sample_column) + COLOR_PRINT_RED_END)
        
        # Enfin, on va vérifier le nombre d'échantillons en fonction des noms de colonnes

samples_list = {}
for c,sample_column in enumerate(sample_columns): # pour chaque nom de colonne
    # on extrait le numéro de l'échantillon du nom de colonne :
    sample_ID = re.sub(column_regex_pattern, "\\1", sample_column)
    
    # et l'identifiant de la piste (ELP, G, A, M, K, L, G-y, A-y, M-y, K-y, L-y)
    track_ID = re.sub(column_regex_pattern, "\\2", sample_column) + re.sub(column_regex_pattern, "\\3", sample_column)
    
    # on stocke le tout dans un dict
    if sample_ID not in samples_list.keys(): # c'est la première colonne que l'on trouve pour cet échantillon
        samples_list[sample_ID] = {"ELP":0,"G":0,"A":0,"M":0,"K":0,"L":0,"G-y":0,"A-y":0,"M-y":0,"K-y":0,"L-y":0}
        
    # on vérifie que l'indice de piste soit bien repértorié (ELP, G, A, M, K, L, G-y, A-y, M-y, K-y, L-y)
    if track_ID not in samples_list[sample_ID].keys():
        print(COLOR_PRINT_RED_START + "Type de piste inconnu pour l'échantillon <{}> : <{}>".format(sample_ID,track_ID) + COLOR_PRINT_RED_END)
        
    # on ajoute donc le compteur pour la piste retrouvée ici
    samples_list[sample_ID][track_ID] += 1
    
# On peut donc vérifier dans notre dict compilé la cohérence des données :

for sample_ID,sample_data in samples_list.items():
    if sample_data != {'ELP': 1, 'G': 1, 'A': 1, 'M': 1, 'K': 1, 'L': 1, 'G-y': 1, 'A-y': 1, 'M-y': 1, 'K-y': 1, 'L-y': 1}:
        print(COLOR_PRINT_RED_START + "Erreur dans les colonnes retrouvées pour l'échantillon <{}> : {}".format(sample_ID,sample_data) + COLOR_PRINT_RED_END)
        
print("Ended columns QC")


Start columns QC
Le jeu de données contient 1803 échantillons
Ended columns QC


### 2.3. Quality check des données échantillons

In [6]:
print("Start samples QC")

possible_labels = ("IgG_L", "IgG_K", "IgA_L", "IgA_K", "IgM_L", "IgM_K", "Complex", "Normal", )

# pour chaque échantillon :
for sample_ID,sample_data in samples_list.items():

    # on récupère la label de l'échantillon et on vérifie qu'elle est conforme aux labels attendues (listées ci-dessus)
    sample_label = raw_dataset.loc[:,sample_ID+"-ELP"].iloc[0]
    if sample_label not in possible_labels:
        print(COLOR_PRINT_RED_START + "Erreur dans la label pour l'échantillon <{}> : <{}> (doit être une parmi : {})".format(sample_ID,sample_label,possible_labels) + COLOR_PRINT_RED_END)

    # on checke que la 1e ligne de G, A, M, K, L est bien vide
    for track in ("G","A","M","K","L"):
        if pd.isna(raw_dataset.loc[:,sample_ID+"-"+track].iloc[0]) == False:
            print(COLOR_PRINT_RED_START + "La première valeur de la colonne {} pour l'échantillon <{}> n'est pas vide".format(track,sample_ID) + COLOR_PRINT_RED_END)

    # on checke que les données de chaque courbe sont bien au format numérique ENTIER
    for track in ("ELP","G","A","M","K","L"):
        if all(raw_dataset.loc[:,sample_ID+"-"+track].iloc[1:].astype(str).str.match("^[0-9]+$")) == False:
            print(COLOR_PRINT_RED_START + "Au moins une valeur de la colonne {} pour l'échantillon <{}> n'est pas au format numérique ENTIER".format(track,sample_ID) + COLOR_PRINT_RED_END)

print("Ended samples QC")


Start samples QC
Ended samples QC
